<a href="https://colab.research.google.com/github/ajinkyabhabad/Algorithmic-Digital-Marketing/blob/master/Olist%20E-Commerce%20Recommendation%20System/3.Recommendation/ALS_Best.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Alternating Least Squares for best customers

ALS (Alternating Least Squares) is an implicit recommendation algorithm to make a recommendation of products to the users. It uses correlation matrix based of product rating provided by customer to recommend the products to customers.

In [1]:
import pandas as pd
import os
import numpy as np 
import random

import random
import pandas as pd
import numpy as np
import pickle
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler

In [2]:
from google.colab import files
uploaded = files.upload()

Saving best.csv to best.csv


In [4]:
import io
Olist_db = pd.read_csv(io.BytesIO(uploaded['best.csv']))

In [5]:
Olist_db


,PRODUCT_ID,ORDER_ID,CUSTOMER_ID,PRODUCT_CATEGORY_NAME,PAYMENT_VALUE,ORDER_DELIVERED_CUSTOMER_DATE,CUSTOMER_UNIQUE_ID,REVIEW_SCORE,CUSTOMER_CITY,RECENCY,FREQUENCY,MONETARY,RANK,R_Quartile,F_Quartile,M_Quartile,RFMScore,Type
0,47859fca9dd7485cbd93c3e8993bb74f,41707ed875aaa6861979c1a2a8dfa9d5,b0ec3894ce1e2d319363928f8b4649a4,cama_mesa_banho,108.70,2018-07-06 14:43:32,a7d4d952c0fcee82d59197e8558d60c1,4,sao paulo,102.0,2,217.40,17979.5,4,4,4,444,Best
1,c12b64bf0861a27e0f06c36ef82f457d,6ee6c06bb9e001463d8eba71bdfc86a7,2c59071cb3b43a971e166dc9e2a410f2,esporte_lazer,162.75,2018-06-26 19:43:48,58cdce1e1299e7d65d1b81a4769e474a,5,vila velha,112.0,2,325.50,9758.0,4,4,4,444,Best
2,bb09cce52b336261572a5a7e25a33795,c985ae8fdce216ddf62df687c3504ecb,9393ea60acdf05a4004483c3b7c24d77,utilidades_domesticas,95.81,2018-05-17 22:51:04,19913c41a2d6a9192020f85be51811e5,5,belo horizonte,152.0,3,287.43,11829.0,4,4,4,444,Best
3,56139431d72cd51f19eb9f7dae4d1617,36750bfd3e70a5f8f95edf2e28f1d60e,352960fe6741c7944bd2007d8d1d3f64,NaN,135.72,2018-07-03 01:58:38,eec8231e8d513be98954dbcbe946528d,5,nova odessa,106.0,2,271.44,12824.0,4,4,4,444,Best
4,dd768d259ee6054e0dadd66c8e2be0b6,05369626de3379a1a26fa24e22048214,d5f6259a2f0fa2907ce0d47876f0852f,cama_mesa_banho,162.85,2018-05-18 22:03:41,6de06281a46d0d459dbe284683e0134c,1,rio de janeiro,151.0,2,325.70,9750.0,4,4,4,444,Best
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,0ac0d30f666402b0d004e26c5e57eb0f,b72a28958acaa4abcff7b24c1b517c42,cb831c015f5b26cae7cda73671c7510f,instrumentos_musicais,221.14,2018-06-21 16:54:32,10b9f2531b9808ef879b829d86792a9f,5,guarulhos,117.0,2,442.28,5881.0,4,4,4,444,Best
1995,00faa46f36261af8bbf3a4d37fa4841b,5355341d3b3fe35edbd80e7c7898551f,8dc9064ed50605925c539815b560c78e,fashion_bolsas_e_acessorios,275.20,2018-06-21 20:03:36,397b44d5bb99eabf54ea9c2b41ebb905,5,niteroi,117.0,2,315.17,10363.0,4,4,4,444,Best
1996,00faa46f36261af8bbf3a4d37fa4841b,5355341d3b3fe35edbd80e7c7898551f,8dc9064ed50605925c539815b560c78e,fashion_bolsas_e_acessorios,39.97,2018-06-21 20:03:36,397b44d5bb99eabf54ea9c2b41ebb905,5,niteroi,117.0,2,315.17,10363.0,4,4,4,444,Best
1997,d7252737cad49d030f9ce83b50249120,93ae3d70a721582a8c0a1d28d804f947,2d994a14e9afaada433bf6e7509a8272,malas_acessorios,207.18,2018-08-09 23:52:32,37d7a8654918a844ec0ce4b049cca9b0,2,contagem,68.0,2,414.36,6564.0,4,4,4,444,Best


In [6]:
features = ['CUSTOMER_ID', 'PRODUCT_ID', 'REVIEW_SCORE']
Olist_db_ = Olist_db[features]
Olist_db_ = Olist_db_.rename(columns={'Review Score': 'Ratings'})
Olist_db_ = Olist_db_.drop_duplicates()

In [7]:
from scipy.sparse import csr_matrix

In [8]:
Olist_db_.CUSTOMER_ID.unique

<bound method Series.unique of 0       b0ec3894ce1e2d319363928f8b4649a4
1       2c59071cb3b43a971e166dc9e2a410f2
2       9393ea60acdf05a4004483c3b7c24d77
3       352960fe6741c7944bd2007d8d1d3f64
4       d5f6259a2f0fa2907ce0d47876f0852f
                      ...               
1993    55297cbeceb1f300c3c3815e1485892d
1994    cb831c015f5b26cae7cda73671c7510f
1995    8dc9064ed50605925c539815b560c78e
1997    2d994a14e9afaada433bf6e7509a8272
1998    c0717bf8d4587e085bfb7bb4e8a52e33
Name: CUSTOMER_ID, Length: 1752, dtype: object>

In [9]:
# pivot ratings into features
df_product_features = Olist_db_.reset_index().pivot_table(
    index='CUSTOMER_ID',
    columns='PRODUCT_ID',
    values='REVIEW_SCORE'
).fillna(0)
# convert dataframe of features to scipy sparse matrix
mat_product_features = csr_matrix(df_product_features.values)

In [10]:
df_product_features.head()

PRODUCT_ID,008cff0e5792219fae03e570f980b330,00d2fa6f2a87633084c8e0400d41a65a,00faa46f36261af8bbf3a4d37fa4841b,011377a7487fef47fc9e73fa5f7322a6,013ee64977aaa6b2b25475095162e0e9,014d94f219fbff1166b9cf700eee36b2,0152f69b6cf919bcdaf117aa8c43e5a2,019003324f0a201dd2c8962beaf94b6d,01b660ebc1a0c293ccf9b117fa6dd8bb,01cf7c4cffff8db0a1cbe612bd2d50a4,01cfc3d9c29504c2d370933e53f002b4,01d40fca05c4227b987ee069be837b49,01d4157fd9738ea81038316ae4cfc192,01e20e6604216c8adb31d463214ba00c,02090f97a28b3d10d19a09f045f6df34,020a80cb704bf65f38761fcedf2af192,027cdd14a677a5834bc67a9789db5021,028b3574d7619f191a7345161ae590e4,02a0c276d5a252215b48269dea73a6e7,02ed3b595447e948efc3eac897e4df0e,0333795efd58e3c33c5850b92721c3e4,03624ffc0958f7ea385b6d4e088532b0,03d817e5e392e78674ed0bd8195f9159,03e931f387c2dd5b006aecdb12515b2e,03f02ee5e8fd233d41dcfe8ad911cf7e,0412ec72192f1d85e92047c0773434ec,0424d4d1508370eb771126734ec9740e,0437850bd55b9b2b9cf104b59a611666,04566e1a1afa83470b25c2642706fc32,045c0f3faa3d81ff1af071e0811b2711,045da764883d50dc071cc70c81226cbc,04ca6bf4b3d9aa78046f0973efe2f4b1,04fd8d64a2539d188bef33bb288410bb,050d890ccc1c0874e166c9af9c6dd821,0515b574204dff0dbdc5dac180f3df3a,052413504e5295cd66ec4ced0b310062,056010e0e6196d99309275e1b2d12c80,058b372f969b59e8c4a27e224243909c,059ca09d32720d9adce3ae12e181339d,05df88a280f9c6edca3b2f5f9b3e75ad,...,fa304c5424fd292aed9c35eb36d9f495,fa6abc53b6abfed3231c37cfa290850a,fa814ebf85f1640b2acea3c4378d40f9,fad3ac91e2ae2e3e0e3f0bed36a565b8,fafe25fbb0bcbd7b5112ab0070940790,fb19ac22e32dfecea21afbdde4b97b5f,fb310009e56c0ab32657c9c674bd402c,fb31d5d2acd4bf37057330c1a96af712,fb4dc3571e0698692224020b5b1b672a,fb55982be901439613a95940feefd9ee,fb8ec7f3124032e812713f60633e6167,fb979d44fd4269b2ea30e300b66c89f3,fbc1488c1a1e72ba175f53ab29a248e8,fbce4c4cb307679d89a3bf3d3bb353b9,fc1d8637c0268af3db482c14b7ef8e75,fc43d4931ce7b45c01dcf5289927dc01,fc5062f483f32b6e7d6e4a38c402d56c,fc5dd987f12a7b823a76a44aa1ba88f6,fc6b299cf83fa76632dd1b0a2c611f33,fc8595227785863846b7daa2ae3f2c9b,fca62108387e25005784da2f551466a1,fcaab5d7f656094e49fbe4ee3a506658,fcd0a70961c103799897803f4e49c628,fcf39794919367e915d2656291f99564,fcf6ad274391aea29f5d6e5ef9da5050,fd0c762860528d119344e1377b574fc5,fd220f6fd5a3f13ef9db07428f9c935d,fdb7db3bff9c924c80ae39b6cd2e82a7,fdbd77fe1773263558126c16e0b4b67c,fde90f06296b26995fd0de6f16610d70,fe705ffbaa3804c5562b8b1046157a6c,fe9c4b2cf9c3adbaf3644880fb72254d,fedbf77eb491dfc9ac276b9ea9ee10c6,fef0dd8c53552d0f784b7cc5375a3ffe,fef7934cc233ee5b1dc13094d98a1465,ff2c1ec09b1bb340e84f0d6b21cc7dbb,ff55834c0b3272853dfe62340f2bb91b,ff7263dfb3cfff5421ada48c3899c313,ff85ff517698c3fe8b200afddda7fb3a,ffb2e8c1ddc7c3e590d2bc4c91de53e1
CUSTOMER_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
00624707c8556199de70cac1593e7088,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00791d8bb3acb245dc0b865656e18fff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00b2b9682f5c2a1f7cf1312ce6a0ea9d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00ca6e4277e535d41bda84d371310be2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [11]:
df_product_features.shape

(873, 1472)

In [12]:
X = df_product_features.T
X.head()

CUSTOMER_ID,00624707c8556199de70cac1593e7088,00791d8bb3acb245dc0b865656e18fff,00b2b9682f5c2a1f7cf1312ce6a0ea9d,00ca6e4277e535d41bda84d371310be2,00fb765118b6d3b18dfc763dc3729a5b,011b97b83f521e755f03173c96baf57b,016890b20bac6cd8155fb02cd3ec3e44,01952039ba4abb6c459e7b6d62a69d7a,0197df52c22d6ef0aa52ffb63ac80404,01aef689891025af290bbdc4788d87da,01ea69b195809e2a8a932de44d8c0f73,0214d81e1cb90cf47f40f5345a1a6c08,02bf19efae533d95c21b5e1ccc633a88,03071a983e944c48990031952443e211,0332d8ec6d259a1caeda045188ffe70f,036a2b2e7d16c12b97890ea51a2ef172,038302481fecf4b95852065a483b8d95,03a62f764ee7e4701929561d653e4029,03b243d7ac6a467982044dd9dddb4d68,03c75f8dfffde486ab30360f6e689a4f,04411d029ba983c1386fc99accec7c63,047965eae6564b51ea05b945966ec595,051471ca58786b55a780eddd05465e89,053773e477b7af0d6678c1669b397cda,0560a02593f282955f85da2db087b078,05c3bc7f2c3c3a45c39595a46c76ad58,062850c890eb64c3bec2243bd551fb8f,06c6f4eaf3c4f75577aced1da71c244f,06d529bccb0825265d8b5b73c66e53d8,07176b181cc48bdaa6807f4a88e17d4a,07276d89be039ef3505f4c6f781bda37,07731871dbc303d9af056c55552492de,07c7a0b82302514ec1c6135dc7dec4cf,07d2fc9925c3b3ef80df52be56a06482,086f2e7a1b67f590d6440b35b2e7246e,08ebd8171843fa54338c3474439144a2,08f632ba547ce8a3f9d4efe3028a216e,097c16abe2faea176ce88673499b5525,0a59f8ef2c08c550b607e7a5b71c754e,0a63548671f07e285558169123b2ab35,...,f2994d32f55503a1bc57258f3d9ecdf6,f31164513a99160d91a464ccf934ed3f,f41d8c045d64733f404e956aef3e569d,f449fd9a4efcb0cea3b97c27c05bae01,f4e2bcb516a63a0b3cf24b4c94ee2209,f52d6c5cdf9a2638cf24e11bd300c158,f5498d05cf105bdd3bc601ff77601d7b,f5b4fe797bf417e8f22de510decbf3ab,f67b9ce7548e47223d43a3ee3242f7d9,f722f83d0923ea785b61ed0cbdf0bc3e,f73d378096054953caa2d69a8abceb73,f7622098214b4634b7fe7eee269b5426,f79c95fa7462b1cb87c31e3b68e30b5f,f79c9ce96b5c0d7c708917616a0ba2fe,f8195b904eae4f051d7ef4511a81e732,f840dfcebe8ead89c4809a63d64b0dbb,f8c39e0e6bfc93d6b12d7ea11a4ecf90,f8cdae87d67f6656d53daa86edc2f488,f8e05a045de09e7855c9df3bb8815e0e,f90b90ce47228752be8ef845070cddca,f90f26ae5ceccfc421ea191d8e72e054,f95848fbfa710dd05f1f05d048be3725,f9fac5d83d911cc24a60b4f7a33433ec,fa3a8736d5ae8f7df9b0a4154d0b14dc,fad67e7a03b7988057f8a83c285f36de,fb6895f16f2b933e56af615dbcecf0aa,fb6966f6b07c089be66da41fcadd4b9d,fbfb33cc34116ccf95b043f7cd31c692,fc2f06b2af43f8be78736cd991ca8486,fc9fdbf5a26b1399e70bb3666cf7f9e6,fd667d6a6784c49bfd41e1ea9e9e6a3e,fdceacbf1e29b39745ee431d0a864d0f,fde106d687f66d052aa9b6b169f250d0,fe154237bdcacd57d9cd2adaa60f1012,fe86c2c022a8d1665da64dabf0ceeaea,fe96dfd861c4c42a3f4d7668a3e22900,febf66a130bc1bc3d35546f466261511,fee8071ecc5ad2f5b2831c4fbd9b3cdd,ffd094af4643c7ac712acd36a56feccc,fff93c1da78dafaaa304ff032abc6205
PRODUCT_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
008cff0e5792219fae03e570f980b330,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00d2fa6f2a87633084c8e0400d41a65a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00faa46f36261af8bbf3a4d37fa4841b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
011377a7487fef47fc9e73fa5f7322a6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [13]:
X.index[:-20]

Index(['008cff0e5792219fae03e570f980b330', '00d2fa6f2a87633084c8e0400d41a65a',
       '00faa46f36261af8bbf3a4d37fa4841b', '011377a7487fef47fc9e73fa5f7322a6',
       '013ee64977aaa6b2b25475095162e0e9', '014d94f219fbff1166b9cf700eee36b2',
       '0152f69b6cf919bcdaf117aa8c43e5a2', '019003324f0a201dd2c8962beaf94b6d',
       '01b660ebc1a0c293ccf9b117fa6dd8bb', '01cf7c4cffff8db0a1cbe612bd2d50a4',
       ...
       'fb8ec7f3124032e812713f60633e6167', 'fb979d44fd4269b2ea30e300b66c89f3',
       'fbc1488c1a1e72ba175f53ab29a248e8', 'fbce4c4cb307679d89a3bf3d3bb353b9',
       'fc1d8637c0268af3db482c14b7ef8e75', 'fc43d4931ce7b45c01dcf5289927dc01',
       'fc5062f483f32b6e7d6e4a38c402d56c', 'fc5dd987f12a7b823a76a44aa1ba88f6',
       'fc6b299cf83fa76632dd1b0a2c611f33', 'fc8595227785863846b7daa2ae3f2c9b'],
      dtype='object', name='PRODUCT_ID', length=1452)

In [14]:
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(1472, 10)

In [15]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(1472, 1472)

In [16]:
correlation_matrix

array([[ 1.        ,  0.68245965,  0.55677242, ..., -0.63199625,
        -0.7120448 ,  0.87388674],
       [ 0.68245965,  1.        ,  0.55738303, ..., -0.73962619,
        -0.32140885,  0.84535255],
       [ 0.55677242,  0.55738303,  1.        , ..., -0.81666394,
        -0.35510888,  0.80259147],
       ...,
       [-0.63199625, -0.73962619, -0.81666394, ...,  1.        ,
         0.48316331, -0.82919022],
       [-0.7120448 , -0.32140885, -0.35510888, ...,  0.48316331,
         1.        , -0.55644027],
       [ 0.87388674,  0.84535255,  0.80259147, ..., -0.82919022,
        -0.55644027,  1.        ]])

In [17]:
fave_prod = Olist_db_.groupby(['CUSTOMER_ID']).max()['PRODUCT_ID'].to_frame()

In [18]:
fave_prod = fave_prod.reset_index()
fave_prod

,CUSTOMER_ID,PRODUCT_ID
0,00624707c8556199de70cac1593e7088,9fab8bd1f9c69cf2301b8ad3bec47480
1,00791d8bb3acb245dc0b865656e18fff,bdef7736f61a507f68b31bd86ddaf69d
2,00b2b9682f5c2a1f7cf1312ce6a0ea9d,5d7c23067ed3fc8c6e699b9373d5890b
3,00ca6e4277e535d41bda84d371310be2,b1dc70b2d3b0d42b355529c4026a35a7
4,00fb765118b6d3b18dfc763dc3729a5b,186fe07c8b8005ec6a498587ffbc1352
...,...,...
868,fe96dfd861c4c42a3f4d7668a3e22900,7c6fbb3a5346dfd607386155c6f628f8
869,febf66a130bc1bc3d35546f466261511,b09790edda54333116f9cf1decc580a1
870,fee8071ecc5ad2f5b2831c4fbd9b3cdd,76397820b3556e99a4a2e3056eca24e5
871,ffd094af4643c7ac712acd36a56feccc,ad21c6ffe97ed6db7a3d73b51bca2ee0


In [19]:
def get_prod_id(customer_id):
    
    product_id = fave_prod[fave_prod.CUSTOMER_ID == customer_id]['PRODUCT_ID']
    return product_id
    
    
prd_id = get_prod_id('fff93c1da78dafaaa304ff032abc6205')
Product_id = prd_id.iloc[0]
Product_id

'3158c44b08596ff51ee3560fad16cc09'

In [20]:
product_names = list(X.index)
product_ID_Idx = product_names.index(Product_id)
product_ID_Idx

305

In [21]:
correlation_product_ID = correlation_matrix[product_ID_Idx]
correlation_product_ID.shape

(1472,)

Recommending top 10 highly coorelated products

In [22]:
Recommend = list(X.index[correlation_product_ID > 0.70])
# Removes the item already bought by the customer
Recommend.remove(Product_id) 
Recommend[0:20]

['0515b574204dff0dbdc5dac180f3df3a',
 '060f0122cdc8f25db4a0d7359fdae7b0',
 '09cb4aa62c518d0842e1c5f28ac940ff',
 '0d85c435fd60b277ffb9e9b0f88f927a',
 '0dce4f5243207cc33533e133b634ae5f',
 '0e314819204e6e8c5f4481cf5e8367a4',
 '17ad136b7421ea8690fded631b651158',
 '186fe07c8b8005ec6a498587ffbc1352',
 '1ae28ef6d0421f92f2e4e6d407e90347',
 '1f78ffcfb88226a0905e335084c1fa5d',
 '208c1210f363c82510f5609d5feb0341',
 '21d2fdced5c69bfc6539b67908a4ad28',
 '2988acfdcc0351677a6a53f4f51b24cf',
 '2b703ec635a9868dc7da64dda15cd828',
 '2f70195374f6c3d3c6542edf5bd17c8c',
 '3c883b9d3fd2d0ce448d4136349dc0a4',
 '42ca857208d059f51c317b503000c385',
 '451482edc91f59c001a5a3a0e4d594bf',
 '4821fcfd5ac4f89ee295f3ac1f20c7a3',
 '577fd304233fe9412349ef6a4a3f6ae6']

In [23]:
## Getting Product names froom prediction 

predictions = pd.DataFrame(Recommend[:20])
predictions.columns = ['Product_ID']
predictions

,Product_ID
0,0515b574204dff0dbdc5dac180f3df3a
1,060f0122cdc8f25db4a0d7359fdae7b0
2,09cb4aa62c518d0842e1c5f28ac940ff
3,0d85c435fd60b277ffb9e9b0f88f927a
4,0dce4f5243207cc33533e133b634ae5f
5,0e314819204e6e8c5f4481cf5e8367a4
6,17ad136b7421ea8690fded631b651158
7,186fe07c8b8005ec6a498587ffbc1352
8,1ae28ef6d0421f92f2e4e6d407e90347
9,1f78ffcfb88226a0905e335084c1fa5d


In [24]:
predictions['Product_ID'] = predictions.Product_ID.apply(lambda x : Olist_db_[Olist_db_.PRODUCT_ID == x]['PRODUCT_ID'].unique()[0])

In [25]:
predictions[:]

,Product_ID
0,0515b574204dff0dbdc5dac180f3df3a
1,060f0122cdc8f25db4a0d7359fdae7b0
2,09cb4aa62c518d0842e1c5f28ac940ff
3,0d85c435fd60b277ffb9e9b0f88f927a
4,0dce4f5243207cc33533e133b634ae5f
5,0e314819204e6e8c5f4481cf5e8367a4
6,17ad136b7421ea8690fded631b651158
7,186fe07c8b8005ec6a498587ffbc1352
8,1ae28ef6d0421f92f2e4e6d407e90347
9,1f78ffcfb88226a0905e335084c1fa5d


In [26]:
Olist_db_.to_csv('Olist_db_ALS.csv', index=False)

In [27]:
X.to_csv('X_ALS.csv')

In [28]:
correlation_matrix

array([[ 1.        ,  0.68245965,  0.55677242, ..., -0.63199625,
        -0.7120448 ,  0.87388674],
       [ 0.68245965,  1.        ,  0.55738303, ..., -0.73962619,
        -0.32140885,  0.84535255],
       [ 0.55677242,  0.55738303,  1.        , ..., -0.81666394,
        -0.35510888,  0.80259147],
       ...,
       [-0.63199625, -0.73962619, -0.81666394, ...,  1.        ,
         0.48316331, -0.82919022],
       [-0.7120448 , -0.32140885, -0.35510888, ...,  0.48316331,
         1.        , -0.55644027],
       [ 0.87388674,  0.84535255,  0.80259147, ..., -0.82919022,
        -0.55644027,  1.        ]])

In [29]:
correlation_matrix.shape

(1472, 1472)

In [30]:
import json

with open('correlation_matrix_ALS.txt', 'w') as filehandle:
    json.dump(correlation_matrix.tolist(), filehandle)

Recommendation Pipeline

In [31]:
Olist_db_ = pd.read_csv('Olist_db_ALS.csv')
Olist_db_

,CUSTOMER_ID,PRODUCT_ID,REVIEW_SCORE
0,b0ec3894ce1e2d319363928f8b4649a4,47859fca9dd7485cbd93c3e8993bb74f,4
1,2c59071cb3b43a971e166dc9e2a410f2,c12b64bf0861a27e0f06c36ef82f457d,5
2,9393ea60acdf05a4004483c3b7c24d77,bb09cce52b336261572a5a7e25a33795,5
3,352960fe6741c7944bd2007d8d1d3f64,56139431d72cd51f19eb9f7dae4d1617,5
4,d5f6259a2f0fa2907ce0d47876f0852f,dd768d259ee6054e0dadd66c8e2be0b6,1
...,...,...,...
1747,55297cbeceb1f300c3c3815e1485892d,71a5f1c2a5fd9889ef26b5ac22aec9c6,5
1748,cb831c015f5b26cae7cda73671c7510f,0ac0d30f666402b0d004e26c5e57eb0f,5
1749,8dc9064ed50605925c539815b560c78e,00faa46f36261af8bbf3a4d37fa4841b,5
1750,2d994a14e9afaada433bf6e7509a8272,d7252737cad49d030f9ce83b50249120,2


In [33]:
X = pd.read_csv('X_ALS.csv', index_col=0)
X

,00624707c8556199de70cac1593e7088,00791d8bb3acb245dc0b865656e18fff,00b2b9682f5c2a1f7cf1312ce6a0ea9d,00ca6e4277e535d41bda84d371310be2,00fb765118b6d3b18dfc763dc3729a5b,011b97b83f521e755f03173c96baf57b,016890b20bac6cd8155fb02cd3ec3e44,01952039ba4abb6c459e7b6d62a69d7a,0197df52c22d6ef0aa52ffb63ac80404,01aef689891025af290bbdc4788d87da,01ea69b195809e2a8a932de44d8c0f73,0214d81e1cb90cf47f40f5345a1a6c08,02bf19efae533d95c21b5e1ccc633a88,03071a983e944c48990031952443e211,0332d8ec6d259a1caeda045188ffe70f,036a2b2e7d16c12b97890ea51a2ef172,038302481fecf4b95852065a483b8d95,03a62f764ee7e4701929561d653e4029,03b243d7ac6a467982044dd9dddb4d68,03c75f8dfffde486ab30360f6e689a4f,04411d029ba983c1386fc99accec7c63,047965eae6564b51ea05b945966ec595,051471ca58786b55a780eddd05465e89,053773e477b7af0d6678c1669b397cda,0560a02593f282955f85da2db087b078,05c3bc7f2c3c3a45c39595a46c76ad58,062850c890eb64c3bec2243bd551fb8f,06c6f4eaf3c4f75577aced1da71c244f,06d529bccb0825265d8b5b73c66e53d8,07176b181cc48bdaa6807f4a88e17d4a,07276d89be039ef3505f4c6f781bda37,07731871dbc303d9af056c55552492de,07c7a0b82302514ec1c6135dc7dec4cf,07d2fc9925c3b3ef80df52be56a06482,086f2e7a1b67f590d6440b35b2e7246e,08ebd8171843fa54338c3474439144a2,08f632ba547ce8a3f9d4efe3028a216e,097c16abe2faea176ce88673499b5525,0a59f8ef2c08c550b607e7a5b71c754e,0a63548671f07e285558169123b2ab35,...,f2994d32f55503a1bc57258f3d9ecdf6,f31164513a99160d91a464ccf934ed3f,f41d8c045d64733f404e956aef3e569d,f449fd9a4efcb0cea3b97c27c05bae01,f4e2bcb516a63a0b3cf24b4c94ee2209,f52d6c5cdf9a2638cf24e11bd300c158,f5498d05cf105bdd3bc601ff77601d7b,f5b4fe797bf417e8f22de510decbf3ab,f67b9ce7548e47223d43a3ee3242f7d9,f722f83d0923ea785b61ed0cbdf0bc3e,f73d378096054953caa2d69a8abceb73,f7622098214b4634b7fe7eee269b5426,f79c95fa7462b1cb87c31e3b68e30b5f,f79c9ce96b5c0d7c708917616a0ba2fe,f8195b904eae4f051d7ef4511a81e732,f840dfcebe8ead89c4809a63d64b0dbb,f8c39e0e6bfc93d6b12d7ea11a4ecf90,f8cdae87d67f6656d53daa86edc2f488,f8e05a045de09e7855c9df3bb8815e0e,f90b90ce47228752be8ef845070cddca,f90f26ae5ceccfc421ea191d8e72e054,f95848fbfa710dd05f1f05d048be3725,f9fac5d83d911cc24a60b4f7a33433ec,fa3a8736d5ae8f7df9b0a4154d0b14dc,fad67e7a03b7988057f8a83c285f36de,fb6895f16f2b933e56af615dbcecf0aa,fb6966f6b07c089be66da41fcadd4b9d,fbfb33cc34116ccf95b043f7cd31c692,fc2f06b2af43f8be78736cd991ca8486,fc9fdbf5a26b1399e70bb3666cf7f9e6,fd667d6a6784c49bfd41e1ea9e9e6a3e,fdceacbf1e29b39745ee431d0a864d0f,fde106d687f66d052aa9b6b169f250d0,fe154237bdcacd57d9cd2adaa60f1012,fe86c2c022a8d1665da64dabf0ceeaea,fe96dfd861c4c42a3f4d7668a3e22900,febf66a130bc1bc3d35546f466261511,fee8071ecc5ad2f5b2831c4fbd9b3cdd,ffd094af4643c7ac712acd36a56feccc,fff93c1da78dafaaa304ff032abc6205
PRODUCT_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
008cff0e5792219fae03e570f980b330,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00d2fa6f2a87633084c8e0400d41a65a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00faa46f36261af8bbf3a4d37fa4841b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
011377a7487fef47fc9e73fa5f7322a6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [34]:
import json
with open('correlation_matrix_ALS.txt') as f:
    correlation_matrix = json.load(f)

In [35]:
correlation_matrix = np.array(correlation_matrix)

In [36]:
#Olist_db_ = pd.read_csv('Olist_db_ALS.csv')
#X = pd.read_csv('X_ALS.csv')
    
#with open('correlation_matrix_ALS.txt') as f:
#    correlation_matrix = json.load(f)

#correlation_matrix = np.array(correlation_matrix)

def Recommendations_ALS(Customer_id):
    #Olist_db_ = Olist_db_[Olist_db_.Customer_ID < 3600]
    fave_prod = Olist_db_.groupby(['CUSTOMER_ID']).max()['PRODUCT_ID'].to_frame()
    fave_prod = fave_prod.reset_index()
    
    #prd_id = get_prod_id(Customer_id)
    prd_id = fave_prod[fave_prod.CUSTOMER_ID == Customer_id]['PRODUCT_ID']
    Product_id = prd_id.iloc[0]
    product_names = list(X.index)
    product_ID_Idx = product_names.index(Product_id)
    
    correlation_product_ID = correlation_matrix[product_ID_Idx]
    
    Recommend = list(X.index[correlation_product_ID > 0.70])
    # Removes the item already bought by the customer
    Recommend.remove(Product_id) 
    
    ## Getting Product names froom prediction 

    predictions = pd.DataFrame(Recommend[:20])
    predictions.columns = ['Product_ID']

    predictions['Product_ID'] = predictions.Product_ID.apply(lambda x : Olist_db_[Olist_db_.PRODUCT_ID == x]['PRODUCT_ID'].unique()[0])
    Recommendations = predictions[:10]
    return Recommendations



In [37]:
Recommendations = Recommendations_ALS('fff93c1da78dafaaa304ff032abc6205')
Recommendations[:10]

,Product_ID
0,0515b574204dff0dbdc5dac180f3df3a
1,060f0122cdc8f25db4a0d7359fdae7b0
2,09cb4aa62c518d0842e1c5f28ac940ff
3,0d85c435fd60b277ffb9e9b0f88f927a
4,0dce4f5243207cc33533e133b634ae5f
5,0e314819204e6e8c5f4481cf5e8367a4
6,17ad136b7421ea8690fded631b651158
7,186fe07c8b8005ec6a498587ffbc1352
8,1ae28ef6d0421f92f2e4e6d407e90347
9,1f78ffcfb88226a0905e335084c1fa5d
